This notebook creates and trains a ResNet model that deals with count-based encoded data.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Possible critical alarm types
critical_alarm_types = [7,15,16,21,33,56,68,95,1000,1001]
df = pd.DataFrame()

for i in critical_alarm_types:
    # Replace the link according to the data you want to read (All, Unique Samples, Random Samples)
    type_df = pd.read_csv("../Data/Final Data/Train/Random Samples/Count-based/" + str(i) + "_countbased.csv")
    df = pd.concat([df, type_df], ignore_index=True)

X = df.drop(columns=['y']).values
y = df['y'].values

If you wish to perform an upsampling using SMOTE, run the following cell. If you do not wish to perform an upsampling, bypass the following cell and run the next one.

In [ ]:
# Upsampling using SMOTE

from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X, y)

In [ ]:
# 1-hot-encoding of the class labels and train-test-split

# Determine the number of unique critical error types
num_classes = len(np.unique(y))
print(num_classes)

# Encode the target variable using LabelEncoder and one-hot encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Creating the ResNet architecture

from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Add, GlobalAveragePooling1D, Dense

def residual_block(input_tensor, filters, kernel_size, strides=1):
    x = Conv1D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Downsample the input tensor if strides=2
    if strides == 2:
        input_tensor = Conv1D(filters, 1, strides=strides, padding='same')(input_tensor)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

# Input layer
inputs = Input(shape=(77, 1))

# Initial convolution
x = Conv1D(64, 7, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Residual blocks
x = residual_block(x, 64, 9)
x = residual_block(x, 64, 9)
x = residual_block(x, 64, 9)

x = residual_block(x, 128, 9, strides=2)
x = residual_block(x, 128, 9)
x = residual_block(x, 128, 9)

x = residual_block(x, 256, 9, strides=2)
x = residual_block(x, 256, 9)
x = residual_block(x, 256, 9)

# Global average pooling
x = GlobalAveragePooling1D()(x)

# Output layer
outputs = Dense(10, activation='softmax')(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss",
                                        mode="min", patience=3,
                                        restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[earlystopping])

In [7]:
# Save model
import joblib

joblib.dump(model, 'ResNet_sequences_all_smote.joblib')

['ResNet_sequences_all_smote.joblib']

In [ ]:
# Prediction with test data

y_pred = model.predict(X_test)

In [ ]:
# Undo 1-hot-encoding of the class labels

import numpy as np

y_pred_classes = []
y_real_classes = []

for item in y_pred:
    y_pred_classes.append(np.argmax(item))

for item in y_test:
    y_real_classes.append(np.argmax(item))

In [ ]:
# Print classification report
#
from sklearn.metrics import classification_report

label_names = ['7', '15', '16', '21', '33', '56', '68', '95', '1000', '1001']
print(classification_report(y_real_classes, y_pred_classes, target_names=label_names))

In [ ]:
# Print confusion matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

label_names = ['7', '15', '16', '21', '33', '56', '68', '95', '1000', '1001']

cm = confusion_matrix(y_real_classes, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_names)
disp.plot()
plt.show()